In [ ]:
!pip install cplex
!pip install docplex

In [ ]:
import pandas as pd
from docplex.cp.model import CpoModel
import numpy as np
import math
import itertools

# Notation and problem statement

- Given two matrices $ A \in \mathbb{R}^{n\times m}$ and $ B \in \mathbb{R}^{m\times p}$, the multiplication of these two matrices result in matrix $ C \in \mathbb{R}^{n\times p}$
- The resulting multiplication tensor $T_n$ defining such matrixmultiplication will be of size $\{0,1\}^{n\times m \times p}$
- Any possible low rank decomposition of $T_n$ will be parametrized by factor matrices U, V and W which have the following sizes:
$$U \in \mathbb{F}^{n \cdot m \times R},V \in \mathbb{F}^{m \cdot p \times R},W \in \mathbb{F}^{n \cdot p \times R}$$
where $\mathbb{F} = \{-1,0,1\}$
---
As per the alphatensor paper notation we characterize different matrix multiplication cases by $(n,m,p)$ and additionally specify a rank R.

## Best known ranks for Square Matrix Multiplication

*   $(n,m,p) = (2,2,2) \ \  \ \ \ $ Best rank R = 7
*   $(n,m,p) = (3,3,3) \ \  \ \ \ $ Best known rank R = 23
*   $(n,m,p) = (4,4,4) \ \  \ \ \ $ Best known rank R = 47
*   $(n,m,p) = (5,5,5) \ \  \ \ \ $ Best known rank R = 96

## Best known ranks for Rectangular Matrix Multiplication

*   $(n,m,p) = (2,2,3) \ \  \ \ \ $ Best known rank R = 11
*   $(n,m,p) = (2,2,4) \ \  \ \ \ $ Best known rank R = 14
*   $(n,m,p) = (2,3,3) \ \  \ \ \ $ Best known rank R = 15
*   $(n,m,p) = (2,2,5) \ \  \ \ \ $ Best known rank R = 18
*   $(n,m,p) = (2,3,4) \ \  \ \ \ $ Best known rank R = 20
*   $(n,m,p) = (2,3,5) \ \  \ \ \ $ Best known rank R = 25
*   $(n,m,p) = (2,4,4) \ \  \ \ \ $ Best known rank R = 26
*   $(n,m,p) = (3,3,4) \ \  \ \ \ $ Best known rank R = 29




In [3]:
def general_multiplication_tensor(N, M, P):
   """Multiplication tensor.
   The multiplication tensor T in {0,1} of size NM x MP x PN
   for the multiplication of two matrices of size NxM and MxP
   """
   T = np.zeros((N * M, M * P, N * P), dtype=np.int64)
   for n in range(N):
       for m in range(M):
           for p in range(P):
               # Convert multi-dimensional indices to flat indices.
               a_index = np.ravel_multi_index((n, m), (N, M))
               b_index = np.ravel_multi_index((m, p), (M, P))
               c_index = np.ravel_multi_index((n, p), (N, P))
               T[a_index, b_index, c_index] = 1
   return T

def expand_pd(U, V, W):
    """Expand a polyadic decomposition.
    The polyadic expansion T of the factor matrices U, V, and W is defined by:
        T[i, j, k] = \sum_r U[i, r] * V[j, r] * W[k, r].
    """
    I, J, K, R = U.shape[0], V.shape[0], W.shape[0], U.shape[1]
    T = np.zeros((I, J, K))
    for i in range(I):
        for j in range(J):
            for k in range(K):
                for r in range(R):
                    T[i, j, k] += U[i, r] * V[j, r] * W[k, r]
    return T

# Constraint Programming Formulations

### Formulation #1: using r as column index for U/V/W

In [4]:
def CP_general_arnaud(N,M,P,T,R,valid_ineq,symmetry,inexact_ineq,seed):
    ## Define variables
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]

    ## Matrix Multiplication as tensor operation
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[i][j][k])

    ## Symmetry
    if symmetry:
                   # 1.1 Permutation Symmetry --- 
                  for r in range(R-1):
                        l1_U =[u[i][r] for i in range(len(u)) ]
                        l1_V=[v[i][r] for i in range(len(v)) ]
                        l2_U=[u[i][r+1] for i in range(len(u)) ]
                        l2_V =[v[i][r+1] for i in range(len(v)) ] 
                        
                        mdl.add(mdl.strict_lexicographic(l1_U + l1_V, l2_U + l2_V))

                  # 1.2 Sign Symmetry --- 
                  for r in range(R):
                        # constraint one set first index 
                        mdl.add(u[0][r]<=0)
                        for i in range(1, N*M):
                            mdl.add(u[i][r] <= mdl.sum(mdl.abs(u[ip][r]) for ip in range(i)))

    ## Valid Inequalities
    if valid_ineq:
                  for r in range(R):
                      #  each U^(r) must have at least one non-zero entry
                      mdl.add(mdl.sum(mdl.abs(u[i][r]) for i in range(len(u)))>=1)
                      #  each V^(r) must have at least one non-zero entry
                      mdl.add(mdl.sum(mdl.abs(v[i][r]) for i in range(len(v)))>=1)
                      #  each W^(r) must have at least one non-zero entry
                      mdl.add(mdl.sum(mdl.abs(w[i][r]) for i in range(len(w)))>=1)

                  # Each output must use at least m of the R terms (inner product requires m)
                  for k in range(len(w)):
                    mdl.add(mdl.sum(mdl.abs(w[k][r]) for r in range(R)) >= M)
                    
                    # Each pair of outputs differ by at least 2 R terms
                    for kp in range(len(w)):
                      if k!=kp:
                        mdl.add(mdl.sum(mdl.abs(w[k][r] - w[kp][r]) for r in range(R))>=2)

                  # Lower bound on the number of active products
                  mdl.add(mdl.sum(mdl.sum(mdl.abs(u[i][r]) for i in range(len(u))) * mdl.sum(mdl.abs(v[j][r]) for j in range(len(v))) for r in range(R)) >= M*N*P)
                  
                  ## TO-DO: Need to double check that this is correct
                  # Each product of the form $A_{i,j}B{j,k} must appear in at least one of the R terms
                  '''for i in range(len(u)):
                    for j in range(len(v)):
                      A_col_idx = (i+1) % M
                      B_row_idx = math.floor((1.0*j)/P)
                      if A_col_idx == B_row_idx:
                        mdl.add(mdl.sum(mdl.abs(u[i][r]*v[j][r]) for r in range(R)) >=1)'''
                        
    ## Inexact Inqualities
    if inexact_ineq:
                pass
    
    
    
    mdl.set_parameters({'LogPeriod': 100000,'RandomSeed':seed})
    #mdl.set_parameters({'LogVerbosity': 'Quiet'})

    msol = mdl.solve()
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

### Formulation #2: using r as row index for U/V/W

In [15]:
def CP_general_elias(N,M,P,T,R,valid_ineq,symmetry,inexact_ineq,seed):
    ## Define variables
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(N*M)] for i in range(R)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(M*P)] for j in range(R)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(N*P)] for k in range(R)]
    
    ## Matrix Multiplication as tensor operation
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                mdl.add(mdl.sum(u[r][i]*v[r][j]*w[r][k] for r in range(R)) == T[i][j][k])
    
    ## Symmetry
    if symmetry:
    
              # 1.1 Permutation Symmetry  ---  Lexicographic Constraint
              for r in range(R-1):       mdl.add(mdl.strict_lexicographic(u[r]+v[r], u[r+1]+v[r+1]))

              # 1.2 Sign Symmetry --- 
              for r in range(R):
                # constraint one set first index 
                mdl.add(u[r][0]<=0)
                for i in range(1, N*M):
                    mdl.add(u[r][i] <= mdl.sum(mdl.abs(u[r][ip]) for ip in range(i)))


    ## Valid Inequalities
    if valid_ineq:
              for r in range(R):
                  #  each U^(r) must have at least one non-zero entry
                  mdl.add(mdl.sum(mdl.abs(u[r][i]) for i in range(len(u[r])))>=1)
                  #  each V^(r) must have at least one non-zero entry
                  mdl.add(mdl.sum(mdl.abs(v[r][i]) for i in range(len(v[r])))>=1)
                  #  each W^(r) must have at least one non-zero entry
                  mdl.add(mdl.sum(mdl.abs(w[r][i]) for i in range(len(w[r])))>=1)


              # Each output must use at least m of the R terms (inner product requires m)
              for k in range(N*P):
                mdl.add(mdl.sum(mdl.abs(w[r][k]) for r in range(R)) >= M)
                # Each pair of outputs differ by at least 2 R terms
                for kp in range(N*P):
                  if k!=kp:
                    mdl.add(mdl.sum(mdl.abs(w[r][k] - w[r][kp]) for r in range(R))>=2)
              # Lower bound on the number of active products
              mdl.add(mdl.sum(mdl.sum(mdl.abs(u[r][i]) for i in range(N*M)) * mdl.sum(mdl.abs(v[r][j]) for j in range(M*P)) for r in range(R)) >= M*N*P)
              # 6) Each product of the form $A_{i,j}B_{j,k}$ must appear in at least one of the $R$ terms  
              for i in range(M*P):
                  for j in range(P*N):
                      A_col_idx = (i+1) % P #GOOD
                      B_row_idx = math.floor((1.0*j)/N)
                      if A_col_idx == B_row_idx:
                        mdl.add(mdl.sum(mdl.abs(u[r][i]*v[r][j]) for r in range(R)) >= 1)
    ## Inexact Inqualities
    if inexact_ineq:
            pass
    
    mdl.set_parameters({'LogPeriod': 100000,'RandomSeed':seed})
  
    #mdl.set_parameters({'LogVerbosity': 'Quiet'})

    msol = mdl.solve()
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

### Cyclic Invariant CP formulation for square matrices

In [6]:
def Cp_opt_cyclic(n,T_n,S,R,valid_ineq):
    T = int((R-S)/3)
    print(T)
  
    mdl = CpoModel()
    A = [[mdl.integer_var(-1, 1, name="A" + str(n) + "_" + str(r)) for r in range(S)] for n in range(n**2)]
    B = [[mdl.integer_var(-1, 1, name="B" + str(n) + "_" + str(r)) for r in range(T)] for n in range(n**2)]
    C = [[mdl.integer_var(-1, 1, name="C" + str(n) + "_" + str(r)) for r in range(T)] for n in range(n**2)]
    D = [[mdl.integer_var(-1, 1, name="D" + str(n) + "_" + str(r)) for r in range(T)] for n in range(n**2)]
    
    for i in range(n**2):
        for j in range(n**2):
            for k in range(n**2):
              r_sum = []
              for r in range(R):
                if r <=S-1:
                 
                  r_sum.append(A[i][r]*A[j][r]*A[k][r])
                elif r>=S and r<=S-1+T:

                    r_sum.append(B[i][r-S]*D[j][r-S]*C[k][r-S])
                elif r>S-2+T and r<=S-1+2*T:
                     r_sum.append(C[i][r-S-T]*B[j][r-S-T]*D[k][r-S-T])
                else:
                  r_sum.append(D[i][r-S-2*T]*C[j][r-S-2*T]*B[k][r-S-2*T])

              mdl.add(mdl.sum(r_sum) == T_n[i][j][k])

    # Valid Inequalities
    if valid_ineq:

          # 1) each U^(r) must have at least one non-zero entry and same for V^(r)
          # and 2) each W_{k} must use at least one of the multiplication terms
          for r in range(T):
                        mdl.add(mdl.sum(mdl.abs(B[i][r]) for i in range(n**2)) >=1)
                        mdl.add(mdl.sum(mdl.abs(D[i][r]) for i in range(n**2)) >=1)
                        mdl.add(mdl.sum(mdl.abs(C[i][r]) for i in range(n**2)) >=1)
          for r in range(S):
                        mdl.add(mdl.sum(mdl.abs(A[i][r]) for i in range(n**2))>=1)
            
          
          for k in range(n**2):
            # 3) Each output must use at least m of the R terms (inner product requires m)
            mdl.add(mdl.sum(mdl.abs(A[k][r]) for r in range(S))+ mdl.sum(mdl.abs(B[k][r]) for r in range(T))+ mdl.sum(mdl.abs(C[k][r]) for r in range(T))+mdl.sum(mdl.abs(D[k][r]) for r in range(T))>=n)
            # 4) Each pair of outputs differ by at least two R terms
            for kp in range(n**2):
              if k != kp:               
                mdl.add(mdl.sum(mdl.abs(A[k][r] - A[kp][r]) for r in range(S))
                        + mdl.sum(mdl.abs(C[k][r] - C[kp][r]) for r in range(T))
                        +mdl.sum(mdl.abs(D[k][r] - D[kp][r]) for r in range(T))
                        + mdl.sum(mdl.abs(B[k][r] - B[kp][r]) for r in range(T))>= 2)
          # 5) Lower bound on the number of active products

          mdl.add( mdl.sum(mdl.sum(mdl.abs(A[i][r]) for i in range(n**2)) * mdl.sum(mdl.abs(A[i][r]) for i in range(n**2)) for r in range(S)) +
                  mdl.sum(mdl.sum(mdl.abs(B[i][r]) for i in range(n**2)) * mdl.sum(mdl.abs(D[i][r]) for i in range(n**2)) for r in range(T)) +
                  mdl.sum(mdl.sum(mdl.abs(C[i][r]) for i in range(n**2)) * mdl.sum(mdl.abs(B[i][r]) for i in range(n**2)) for r in range(T)) +
                  mdl.sum(mdl.sum(mdl.abs(D[i][r]) for i in range(n**2)) * mdl.sum(mdl.abs(C[i][r]) for i in range(n**2)) for r in range(T)) >=n*n*n)
          
          # 6) Each product of the form $A_{i,j}B_{j,k}$ must appear in at least one of the $R$ terms  
          '''for i in range(n**2):
                for j in range(n**2):
                    A_col_idx = (i+1) % n #GOOD
                    B_row_idx = math.floor((1.0*j)/n)
                    if A_col_idx == B_row_idx:


                      mdl.add(
                            mdl.sum(mdl.abs(A[i][r]*A[j][r]) for r in range(S))+
                            mdl.sum(mdl.abs(B[i][r]*D[j][r]) for r in range(T))+
                            mdl.sum(mdl.abs(C[i][r]*B[j][r]) for r in range(T))+
                            mdl.sum(mdl.abs(D[i][r]*C[j][r]) for r in range(T))>= 1)'''


    mdl.set_parameters({'LogPeriod': 100000,'TimeLimit':600})
   
    msol = mdl.solve()
    if msol:
      return [msol,A,B,C,D]
    else:
      print("Infeasible")
      return -1

### Random seed checker

In [7]:
def CP_general_seed(N,M,P,T,R,time_limit,num_seeds):
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]

    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[i][j][k])

    mdl.set_parameters({'LogPeriod': 100000,'TimeLimit':time_limit})
    print("\n \n \n Is the model a satisifaction problem -->  ",mdl.is_satisfaction())
    print("Model Info")
    mdl.print_information()
    print("Starting to solve")
    mdl.run_seeds(num_seeds)

# Experiments


### Searching for SAT (currently attainable ranks w/ CP formulation)



*   2,2,2 R = 7
*   2,2,3 R = 11



In [10]:
cases = [ [[2,2,2],7]   , [[2,2,3],11]]


for case_ in cases:
  R = case_[1]
  N,M,P = case_[0]
  print(R)
  print("\n\n\n\n\n\n")
  print("\n\n\n\n\n\n")
  print("\n\n\n\n\n\n")
  T_n = general_multiplication_tensor(N,M,P)
  print(T_n)

  solution = CP_general_arnaud(N,M,P,T_n,R,False,False,False,seed = 4)

  if type(solution)!=int:
    sol,U,V,W = solution
    U_sol = np.zeros((M*N,R))
    V_sol = np.zeros((P*M,R))
    W_sol = np.zeros((P*N,R))
    
    for i in range(M*N):
      for r in range(R):        U_sol[i,r] = sol[U[i][r]]
    for i in range(P*M):
      for r in range(R):        V_sol[i,r] = sol[V[i][r]]
    for i in range(P*N):
      for r in range(R):        W_sol[i,r] = sol[W[i][r]]
      
    print(U_sol)
    print(V_sol)
    print(W_sol)

    print("\n\n\n\n\n\n")
    t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

    print(T_n)
    print(t_constraint_programming)
    if (t_constraint_programming==T_n).all():
      print("Holy **** CP got the correct T_n for non square matrices")
  else:
    print("Infeasible")

7





















[[[1 0 0 0]
  [0 1 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 1 0 0]]

 [[0 0 1 0]
  [0 0 0 1]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 1 0]
  [0 0 0 1]]]
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 84 variables, 64 constraints
 ! LogPeriod            = 100000
 ! RandomSeed           = 4
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 133.1 (before), 133.1 (after)
 !  . Memory usage      : 495.8 kB (before), 495.8 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
                     100k          6    1         1 != V1_0
                     100k          7    2         1 != U1_0
 *                   117k  1.28s        2         1  = V2_5
 ! ----------------------

### Searching for UNSAT


In [ ]:
dict_ = {1:'base CP formulation',
         2:'base CP formulation + valid_ineq',
         3:'base CP formulation + valid_ineq + symmetry'}

N = 2
M = 2
P = 2

T_n = general_multiplication_tensor(N,M,P)
print(T_n)
print("\n\n\n\n\n\n")
print("Matrix mult for N,M by M,P matrices")
print("N=   ",N)
print("M=   ",M)
print("P=   ",P)
print("R=   ",R)
print()
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
for R in [6]:


  for i in [3]:
    mode = dict_[i]
    
    print("\n\n\n\n\n\n")
    print("Current solving using this mode = ",mode)
    print("For this rank R = ",R)
    print("\n\n\n\n\n\n")
    print("\n\n\n\n\n\n")
    print("\n\n\n\n\n\n")
    
    if i == 1:  solution = CP_general_elias(N,M,P,T_n,R,valid_ineq = False,symmetry= False,inexact_ineq= False,seed = 4)

    if i == 2:  solution = CP_general_elias(N,M,P,T_n,R,valid_ineq = False,symmetry= True,inexact_ineq= False,seed = 4)

    if i == 3:  solution = CP_general_elias(N,M,P,T_n,R,valid_ineq = True,symmetry= True,inexact_ineq= False,seed = 4)

    if type(solution)!=int:
        sol,U,V,W = solution
        U_sol = np.zeros((M*P,R))
        V_sol = np.zeros((P*N,R))
        W_sol = np.zeros((M*N,R))

        for i in range(M*P):
          for r in range(R):
            U_sol[i,r] = sol[U[r][i]]
        for i in range(P*N):
          for r in range(R):
            V_sol[i,r] = sol[V[r][i]]
        for i in range(M*N):
          for r in range(R):
            W_sol[i,r] = sol[W[r][i]]
          
        print(U_sol)
        print(V_sol)
        print(W_sol)

        print("\n\n\n\n\n\n")
        t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

        print(T_n)
        print(t_constraint_programming)
        if (t_constraint_programming==T_n).all():
          print("Holy ** CP got the correct T_n for non square matrices")
    else:
      print("\n \n \n Infeasible \n \n \n")

[[[1 0 0 0]
  [0 1 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 1 0 0]]

 [[0 0 1 0]
  [0 0 0 1]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 1 0]
  [0 0 0 1]]]







Matrix mult for N,M by M,P matrices
N=    2
M=    2
P=    2
R=    5





























Current solving using this mode =  base CP formulation + valid_ineq + symmetry
For this rank R =  6





















 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 72 variables, 136 constraints
 ! LogPeriod            = 100000
 ! RandomSeed           = 4
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 110.6 (before), 110.6 (after)
 !  . Memory usage      : 575.6 kB (before), 575.6 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
  

In [16]:
dict_ = {1:'base CP formulation',
         2:'base CP formulation + valid_ineq',
         3:'base CP formulation + valid_ineq + symmetry'}

N = 2
M = 2
P = 2

T_n = general_multiplication_tensor(N,M,P)
print(T_n)
print("\n\n\n\n\n\n")
print("Matrix mult for N,M by M,P matrices")
print("N=   ",N)
print("M=   ",M)
print("P=   ",P)
print("R=   ",R)
print()
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
for R in [1,2,3,4,5]:


  for i in [3]:
    mode = dict_[i]
    
    print("\n\n\n\n\n\n")
    print("Current solving using this mode = ",mode)
    print("For this rank R = ",R)
    print("\n\n\n\n\n\n")
    print("\n\n\n\n\n\n")
    print("\n\n\n\n\n\n")
    
    if i == 1:  solution = CP_general_elias(N,M,P,T_n,R,valid_ineq = False,symmetry= False,inexact_ineq= False,seed = 4)

    if i == 2:  solution = CP_general_elias(N,M,P,T_n,R,valid_ineq = False,symmetry= True,inexact_ineq= False,seed = 4)

    if i == 3:  solution = CP_general_elias(N,M,P,T_n,R,valid_ineq = True,symmetry= True,inexact_ineq= False,seed = 4)

    if type(solution)!=int:
        sol,U,V,W = solution
        U_sol = np.zeros((M*P,R))
        V_sol = np.zeros((P*N,R))
        W_sol = np.zeros((M*N,R))

        for i in range(M*P):
          for r in range(R):
            U_sol[i,r] = sol[U[r][i]]
        for i in range(P*N):
          for r in range(R):
            V_sol[i,r] = sol[V[r][i]]
        for i in range(M*N):
          for r in range(R):
            W_sol[i,r] = sol[W[r][i]]
          
        print(U_sol)
        print(V_sol)
        print(W_sol)

        print("\n\n\n\n\n\n")
        t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

        print(T_n)
        print(t_constraint_programming)
        if (t_constraint_programming==T_n).all():
          print("Holy ** CP got the correct T_n for non square matrices")
    else:
      print("\n \n \n Infeasible \n \n \n")

[[[1 0 0 0]
  [0 1 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 1 0 0]]

 [[0 0 1 0]
  [0 0 0 1]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 1 0]
  [0 0 0 1]]]







Matrix mult for N,M by M,P matrices
N=    2
M=    2
P=    2
R=    5





























Current solving using this mode =  base CP formulation + valid_ineq + symmetry
For this rank R =  1





















<ipython-input-15-c328deea878a>:41(stream:102:18): Warning: The constraint is always false, the model is infeasible.
                                                              sum([abs(W0_0)]) >= 2
<ipython-input-15-c328deea878a>:41(stream:108:18): Warning: The constraint is always false, the model is infeasible.
                                                              sum([abs(W0_1)]) >= 2
<ipython-input-15-c328deea878a>:41(stream:114:18): Warning: The constraint is always false, the model is infeasible.
                                                        

### Effect of Random seed checker

In [ ]:
R=7
print(R)
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")
print("\n\n\n\n\n\n")

N = 2
M = 2
P = 2
T_n = general_multiplication_tensor(N,M,P)
print(T_n)

solution = CP_general_seed(N,M,P,T_n,R,time_limit = 800 , num_seeds = 10)
